In [12]:
%env SERPER_API_KEY=YOUR_SERPER_API_KEY
%env OPENAI_API_KEY=YOUR_OPENAI_API_KEY

env: SERPER_API_KEY=YOUR_SERPER_API_KEY
env: OPENAI_API_KEY=YOUR_OPENAI_API_KEY


In [5]:
import os
import dspy

from dspy.predict.avatar import Avatar

dspy.settings.configure(
    lm=dspy.OpenAI(
        model="gpt-3.5-turbo-0125",
        api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=4000,
    )
)

class QASignature(dspy.Signature):
    """You will be given a question. Your task is to answer the question."""
    
    question: str = dspy.InputField(
        prefix="Question:",
        desc="question to ask",
        format=lambda x: x.strip(),
    )
    answer: str = dspy.OutputField(
        prefix="Answer:",
        desc="answer to the question",
    )

In [8]:
from dspy.predict.avatar import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper, ArxivAPIWrapper

tools = [
    Tool(
        tool=GoogleSerperAPIWrapper(),
        name="WEB_SEARCH",
        desc="Search the web for information to help answer the question.",
    ),
    Tool(
        tool=ArxivAPIWrapper(),
        name="ARXIV_SEARCH",
        desc="Search the doi through arxiv to get the paper."
    ),
]

avatar = Avatar(
    QASignature,
    tools=tools,
    verbose=True,
)

In [9]:
answer = avatar(question="Give me a brief overview of the history of the internet.").answer

print(f"\n\nAnswer:\n{answer}")

Starting the task...
Action 1: WEB_SEARCH (History of the internet)
Action 2: Finish (No additional input required)


Answer:
The Internet started in the 1960s as a way for government researchers to share information. It has its origins in the efforts of scientists and engineers to build and interconnect computer networks.


In [10]:
answer = avatar(question="Summarize 2201.00432 for me and share titles for related papers on web.").answer
      
print(f"\n\nAnswer:\n{answer}")

Starting the task...
Action 1: ARXIV_SEARCH (2201.00432)
Action 2: WEB_SEARCH (titles related to independence polynomials of trees)
Action 3: Finish (No further action needed.)


Answer:
The paper 2201.00432 is titled "A Linear Algorithm for Computing Independence Polynomials of Trees" by Ohr Kadrawi, Vadim E. Levit, Ron Yosef, Matan Mizrachi. The paper discusses an efficient algorithm for computing the independence polynomials of trees in linear time without the need for a database. Related papers on the topic include "On Unimodality of Independence Polynomials of Trees" and "An identity for the independence polynomials of trees" by Ivan Gutman.
